# Assignment #2: Language models
Author: Pierre Nugues

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment

## Submission

Once you have written all the missing code and run all the cells, you will show it to an instructor that will pass you.

## Organization

* Each group will have to write Python programs to count unigrams, bigrams, and trigrams in a corpus of approximately one million words and to determine the probability of a sentence.
* You can test you regular expression using the regex101.com site
* Each student will have to write a short report of one to two pages and comment briefly the results. In your report, you must produce the tabulated results of your analysis as described below.

## Programming

### Imports

Some imports you may need. Add others as needed.

In [2]:
import bz2
import math
import os
import regex as re
import requests
import sys
from zipfile import ZipFile

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

In [3]:
# You may have to adjust the path
corpus = open('corpus/Selma.txt', encoding='utf8').read()

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [4]:
pattern = 'Nils Holgersson'
width = 25

In [5]:
# spaces match tabs and newlines
pattern = re.sub(' ', r'\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub(r'\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, clean_corpus):
    print(match.group(1))
# print the string with 0..width characters on either side

Selma Lagerlöf Nils Holgerssons underbara resa genom Sv
! Se på Tummetott! Se på Nils Holgersson Tummetott!» Genast vände
r,» sade han. »Jag heter Nils Holgersson och är son till en husma
lden. »Inte är det värt, Nils Holgersson, att du är ängslig eller
 i dem. På den tiden, då Nils Holgersson drog omkring med vildgäs
ulle allt visa honom vad Nils Holgersson från Västra Vemmenhög va
om ägde rum det året, då Nils Holgersson for omkring med vildgäss
m vad det kan kosta dem. Nils Holgersson hade inte haft förstånd 
de det inte mer sägas om Nils Holgersson, att han inte tyckte om 
 Rosenbom?» För där stod Nils Holgersson mitt uppe på Rosenboms n
 Med ens fingo de syn på Nils Holgersson, och då sköt den store v
vila. När vildgässen och Nils Holgersson äntligen hade letat sig 
 slags arbetare. Men vad Nils Holgersson inte såg, det var, att s
nde han fråga, och om då Nils Holgersson sade nej, började han ge
de lille Mats, och om nu Nils Holgersson också hade tegat, så had
åg så försmädlig ut,

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [6]:
def tokenize(text):
    words = re.findall(r'\p{L}+', text)
    return words

In [7]:
words = tokenize(corpus)
words[:10]

['Selma',
 'Lagerlöf',
 'Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Första',
 'bandet']

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [8]:
# Write your code here
words_set = set(words)

print(len(words_set))

44256


Lowercased text

In [9]:
# Write your code here
words_set_lowercase = set(map(lambda w: w.lower(), words_set))

print(len(words_set_lowercase))#Ska vara 41032

41032


### Segmenting a corpus

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

Write a regular expression that matches all the characters that are neither a letter nor a punctuation sign. The punctuations signs will be the followings: `.;:?!`. In your regex, use the same order. For the definition of a letter, use a Unicode regex. You will call the regex string `nonletter`

In [10]:
# Write your code
nonletter = r"[^\p{L}.:;?!]+"


Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [11]:
# Write your code here
def clean(text):
    return re.sub(nonletter, " ", text)

print(clean("En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa."))
 

En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.


In [12]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

In [13]:
test_para = clean(test_para)
test_para

'En gång hade de på Mårbacka en barnpiga som hette Back Kajsa. Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.'

In [14]:
print(corpus[:200])

Selma Lagerlöf


Nils Holgerssons underbara resa genom Sverige. Första bandet





Bokutgåva


Albert Bonniers förlag, Stockholm 1907.




DEN KRISTLIGA DAGVISAN.





Den signade dag, som vi nu här s


In [15]:
print(clean(corpus[:200]))

Selma Lagerlöf Nils Holgerssons underbara resa genom Sverige. Första bandet Bokutgåva Albert Bonniers förlag Stockholm . DEN KRISTLIGA DAGVISAN. Den signade dag som vi nu här s


#### Segmenter

In this section, you will write a sentence segmenter that will delimit each sentence with `</s>` and `<s>` symbols. For example the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

will be bracketed as:

`<s> En gång hade de på Mårbacka en barnpiga som hette Back-Kajsa </s>`

As algorithm, you will use a simple heuristics to detect the sentence boundaries: A sentence starts with a capital letter and ends with a period-equivalent punctuation sign. You will write a regex to match these boundaries with a regular expression and you will insert `</s>\n<s>` symbols with a substitution function.

##### Detecting sentence boundaries

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces. Do not use `\s`.

In [16]:
# Write your code here
sentence_boundaries = r"(\p{P})(\p{Z}+)(\p{Lu})"


##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [17]:
# Write your code here
sentence_markup = r" </s>\n<s> \3"


##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

In [18]:
# Write your code here
text = re.sub(sentence_boundaries, sentence_markup, test_para)
print(text)

En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.


In [19]:
print(text)

En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.


Insert markup codes in the beginning and end of the text

In [20]:
# Write your code here
text = re.sub(r"([\s\S]+)", r"<s> \1 </s>", text)

print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden. </s>


In [21]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden. </s>


Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex. Do not use `\s`.

In [22]:
# Write your code here
text = re.sub(r"(\p{Z}+)", " ", text)
text = re.sub(r"\.", "", text)

In [23]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [24]:
# Write your code here
def segment_sentences(text):
    text = re.sub(sentence_boundaries, sentence_markup, text)
    text = re.sub(r"([\s\S]+)", r"<s> \1 </s>", text)
    text = re.sub(r"(\p{Z}+)", " ", text)
    text = re.sub(r"\.", "", text)
    text = text.lower()
    return text


In [25]:
print(segment_sentences(test_para))

<s> en gång hade de på mårbacka en barnpiga som hette back kajsa </s>
<s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Estimate qualitatively the accuracy of your program.

#### Tokenizing the corpus

Clean and segment the corpus using the functions you have written

In [26]:
# Write your code here
corpus = segment_sentences(clean_corpus)
print(corpus[-565:])

<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste, och sådan kärlek måste vändas i välsignelse </s>
<s> då prästen sade detta, kom alla människor att se bort mot klara gulla, och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka, hon, som var uppkallad efter själva solen, vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen, då hon gick till kyrkan i den röda klänningen, om inte vackrare </s>


The result should be a normalized text without punctuation signs where all the sentences are delimited with `<s>` and `</s>` tags. The five last lines of the text should look like this. You may have some small differences.

In [27]:
print(corpus[-565:])

<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste, och sådan kärlek måste vändas i välsignelse </s>
<s> då prästen sade detta, kom alla människor att se bort mot klara gulla, och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka, hon, som var uppkallad efter själva solen, vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen, då hon gick till kyrkan i den röda klänningen, om inte vackrare </s>


You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [28]:
# Write your code here
words = re.split(r"[ \n]+", corpus)


The five last lines of the corpus should like this:

In [29]:
print(words[-101:])

['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste,', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta,', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla,', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka,', 'hon,', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen,', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen,', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen,', 'om', 'inte', 'vackrare', '</s>']


### Counting unigrams and bigrams

Read and try programs to compute the frequency of unigrams and bigrams of the training set: [<a
            href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>].

#### Unigrams

In [30]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency

We compute the frequencies.

In [31]:
frequency = unigrams(words)
list(frequency.items())[:20]

[('<s>', 51406),
 ('selma', 40),
 ('lagerlöf', 221),
 ('nils', 85),
 ('holgerssons', 6),
 ('underbara', 22),
 ('resa', 271),
 ('genom', 687),
 ('sverige', 32),
 ('</s>', 51406),
 ('första', 488),
 ('bandet', 5),
 ('bokutgåva', 11),
 ('albert', 15),
 ('bonniers', 11),
 ('förlag', 11),
 ('stockholm', 62),
 ('1907', 2),
 ('den', 10689),
 ('kristliga', 2)]

#### Bigrams

In [32]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams

In [33]:
frequency_bigrams = bigrams(words)
list(frequency_bigrams.items())[:20]

[(('<s>', 'selma'), 10),
 (('selma', 'lagerlöf'), 8),
 (('lagerlöf', 'nils'), 1),
 (('nils', 'holgerssons'), 6),
 (('holgerssons', 'underbara'), 4),
 (('underbara', 'resa'), 4),
 (('resa', 'genom'), 6),
 (('genom', 'sverige'), 4),
 (('sverige', '</s>'), 14),
 (('</s>', '<s>'), 51405),
 (('<s>', 'första'), 9),
 (('första', 'bandet'), 1),
 (('bandet', 'bokutgåva'), 2),
 (('bokutgåva', 'albert'), 11),
 (('albert', 'bonniers'), 11),
 (('bonniers', 'förlag'), 11),
 (('förlag', '</s>'), 11),
 (('<s>', 'stockholm'), 13),
 (('stockholm', '1907'), 2),
 (('1907', '</s>'), 2)]

In the report, tell what is the possible number of bigrams and their real number? Explain why such a difference. What would be the possible number of 4-grams.

Propose a solution to cope with bigrams unseen in the corpus.

### Computing the likelihood of a sentence
You will now compute the likelihood of a sentence using a bigram and a trigram models. 

In both models, you will ignore the start of sentence symbol, `<s>`, as this factor is common to both models: $P(<s>)$. This will save you one multiplication.

#### Unigrams

Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [<a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>]. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_.

Your figures might be slightly different because of differences in the sentence segmentation.

```
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================
det 	 21108 	 1041631 	 0.0202643738521607
var 	 12090 	 1041631 	 0.01160679741674355
en 	 13514 	 1041631 	 0.01297388422579589
gång 	 1332 	 1041631 	 0.001278763784871994
en 	 13514 	 1041631 	 0.01297388422579589
katt 	 16 	 1041631 	 1.5360525944408337e-05
som 	 16288 	 1041631 	 0.015637015411407686
hette 	 97 	 1041631 	 9.312318853797554e-05
nils 	 87 	 1041631 	 8.352285982272032e-05
</s> 	 59047 	 1041631 	 0.056687060964967444
=====================================================
Prob. unigrams:	 5.361459667285409e-27
Geometric mean prob.: 0.0023600885848765307
Entropy rate:	 8.726943273141258
Perplexity:	 423.71290908655254
```

In [34]:
# Write your code
def unigram_lm(frequency, sent_words):
    num_words = len([w for w in words if w != "<s>"])
    #print(len(symbols_removed))#Ska bli 1041631 men blir 977206
    Pwi = dict()
    entropy_rates = []

    print('''
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================''')
    prob_unigrams = 1
    
    for word in sent_words:
        P = frequency[word]/num_words #P = pwi_value
        Pwi.update({word: P})
        prob_unigrams *= P
        H = -P * math.log2(P) #H(x) = -P(x) * log2(P(x)) #H = word_entropy_rate
        entropy_rates.append(H)
        print(f"{word} \t {frequency[word]} \t {num_words} \t {Pwi[word]}")
    
    
    print("=====================================================")
    print(f"Prob. unigrams:\t{prob_unigrams}") #Prob. unigrams:	 5.361459667285409e-27

    geometric_mean_prob = prob_unigrams ** (1/len(sent_words))
    print(f"Geometric mean prob.:\t{geometric_mean_prob}") #Geometric mean prob.: 0.0023600885848765307
    
    #entropy_rate = sum(entropy_rates)/len(sent_words) #-log2 istället
    entropy_rate = -math.log2(geometric_mean_prob) #-log2 istället
    print(f"Entropy rate:\t{entropy_rate}")#Entropy rate:	 8.726943273141258
    #Denna blir megafel fråga varför
    
    perplexity = 2**(- (1/len(sent_words) * math.log2(prob_unigrams)))
    #Perplexity(S) = 2^(- (1 / N) * log2(P(S))), where N = #words in sentence, P(S) = product of probabilities in sentence
    print(f"Perplexity:\t{perplexity}")#Perplexity:	 423.71290908655254

    return perplexity


unigram_lm(frequency, 'det var en gång en katt som hette nils </s>'.split())


wi 	 C(wi) 	 #words 	 P(wi)
det 	 18649 	 977206 	 0.019084000712234678
var 	 11511 	 977206 	 0.011779501967855294
en 	 13269 	 977206 	 0.013578508523279636
gång 	 1147 	 977206 	 0.001173754561474244
en 	 13269 	 977206 	 0.013578508523279636
katt 	 9 	 977206 	 9.209931171114382e-06
som 	 16262 	 977206 	 0.016641322300518006
hette 	 90 	 977206 	 9.209931171114381e-05
nils 	 85 	 977206 	 8.698268328274694e-05
</s> 	 51406 	 977206 	 0.052605080198033984
Prob. unigrams:	3.142236532820061e-27
Geometric mean prob.:	0.0022372982728021762
Entropy rate:	8.804026677792676
Perplexity:	446.96767174790574


446.96767174790574

In [35]:
sentence = 'det var en gång en katt som hette nils </s>'
sent_words = sentence.split()
sent_words

['det', 'var', 'en', 'gång', 'en', 'katt', 'som', 'hette', 'nils', '</s>']

In [36]:
perplexity_unigrams = unigram_lm(frequency, sent_words)


wi 	 C(wi) 	 #words 	 P(wi)
det 	 18649 	 977206 	 0.019084000712234678
var 	 11511 	 977206 	 0.011779501967855294
en 	 13269 	 977206 	 0.013578508523279636
gång 	 1147 	 977206 	 0.001173754561474244
en 	 13269 	 977206 	 0.013578508523279636
katt 	 9 	 977206 	 9.209931171114382e-06
som 	 16262 	 977206 	 0.016641322300518006
hette 	 90 	 977206 	 9.209931171114381e-05
nils 	 85 	 977206 	 8.698268328274694e-05
</s> 	 51406 	 977206 	 0.052605080198033984
Prob. unigrams:	3.142236532820061e-27
Geometric mean prob.:	0.0022372982728021762
Entropy rate:	8.804026677792676
Perplexity:	446.96767174790574


In [37]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams

446

#### Bigrams

Write a program to compute the sentence probability using bigrams. Your function will tabulate and print the results as below. It will return the perplexity.

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [38]:
# Write your code
def bigram_lm(frequency, frequency_bigrams, sent_words):
    num_words = len([w for w in words if w != "<s>"])
    with_start_symbol = ["<s>"] + [w for w in sent_words if w != "</s>"]
    prob_bigrams = 1
    entropy_rates = []
    print('''
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================''')
    for index in range(len(sent_words)):
        wi = with_start_symbol[index]
        wi_plus_1 = sent_words[index]
        backoff = False
        
        Ci = frequency.get(wi)
        Ci_i_plus_1 = frequency_bigrams.get((wi, wi_plus_1), 0)

        if Ci_i_plus_1 == 0:
            backoff = True
            P_wi_plus_1_wi = frequency[wi_plus_1]/num_words
            #P_wi_plus_1_wi = frequency.get(wi_plus_1, 0)/num_words #Använd om något ord inte finns alls men just nu skiter jag i det
        else:
            P_wi_plus_1_wi = Ci_i_plus_1/Ci

        if backoff:
            print(f"{wi}\t{wi_plus_1}\t{Ci_i_plus_1}\t{Ci}\t{0}\t*backoff:\t{P_wi_plus_1_wi}")
        else:
            print(f"{wi}\t{wi_plus_1}\t{Ci_i_plus_1}\t{Ci}\t{P_wi_plus_1_wi}")


        prob_bigrams *= P_wi_plus_1_wi

        H = -P_wi_plus_1_wi * math.log2(P_wi_plus_1_wi)
        entropy_rates.append(H)

        

    print("=====================================================")
    print(f"Prob. bigrams:\t{prob_bigrams}") #Prob. bigrams:	 2.376007803503683e-19
    
    geometric_mean_prob = prob_bigrams ** (1/len(sent_words))
    print(f"Geometric mean prob.:\t{geometric_mean_prob}") #Geometric mean prob.: 0.013727289294133601

    entropy_rate = sum(entropy_rates)/len(sent_words)
    entropy_rate = -math.log2(geometric_mean_prob) #-log2 istället
    print(f"Entropy rate:\t{entropy_rate}")#Entropy rate:	 6.186809422848149

    perplexity = 2**(- (1/len(sent_words) * math.log2(prob_bigrams)))
    #Perplexity(S) = 2^(- (1 / N) * log2(P(S))), where N = #words in sentence, P(S) = product of probabilities in sentence
    print(f"Perplexity:\t{perplexity}")#Perplexity:	 72.84759420254609
    
    
    return perplexity

perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words)


wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
<s>	det	4519	51406	0.08790802630043186
det	var	3359	18649	0.18011689634832967
var	en	706	11511	0.06133263834592998
en	gång	601	13269	0.045293541336950786
gång	en	20	1147	0.017436791630340016
en	katt	3	13269	0.00022609088853719196
katt	som	0	9	0	*backoff:	0.016641322300518006
som	hette	45	16262	0.0027671873078342147
hette	nils	0	90	0	*backoff:	8.698268328274694e-05
nils	</s>	1	85	0.011764705882352941
Prob. bigrams:	8.171481688458196e-21
Geometric mean prob.:	0.009800090385646538
Entropy rate:	6.672989229482946
Perplexity:	102.03987521019454


In [39]:
perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words)


wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
<s>	det	4519	51406	0.08790802630043186
det	var	3359	18649	0.18011689634832967
var	en	706	11511	0.06133263834592998
en	gång	601	13269	0.045293541336950786
gång	en	20	1147	0.017436791630340016
en	katt	3	13269	0.00022609088853719196
katt	som	0	9	0	*backoff:	0.016641322300518006
som	hette	45	16262	0.0027671873078342147
hette	nils	0	90	0	*backoff:	8.698268328274694e-05
nils	</s>	1	85	0.011764705882352941
Prob. bigrams:	8.171481688458196e-21
Geometric mean prob.:	0.009800090385646538
Entropy rate:	6.672989229482946
Perplexity:	102.03987521019454


In [40]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

102

In addition to this sentence, _Det var en gång en katt som hette Nils_, write three other sentences that will form your test set and run your programs on them. You will insert them in your report.

In [41]:
sentence1 = "fyra sjöar finns runt staden </s>"
sentence2 = "det är också tillåtet om du vill söka olika kurser </s>"
sentence3 = "man säger vad roligt att träffas kramar om varandra och allt känns tryggt och roligt </s>"

bigram_lm(frequency, frequency_bigrams, sentence1.split())
bigram_lm(frequency, frequency_bigrams, sentence2.split())
bigram_lm(frequency, frequency_bigrams, sentence3.split())



wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
<s>	fyra	1	51406	1.9452982142162394e-05
fyra	sjöar	0	88	0	*backoff:	2.6606467827663767e-05
sjöar	finns	0	26	0	*backoff:	0.0003837471321297659
finns	runt	0	375	0	*backoff:	0.00014019561893807448
runt	staden	0	137	0	*backoff:	0.0001381489675667157
staden	</s>	50	135	0.37037037037037035
Prob. bigrams:	1.424744128456685e-21
Geometric mean prob.:	0.0003354461507041241
Entropy rate:	11.541631190910675
Perplexity:	2981.104412439767

wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
<s>	det	4519	51406	0.08790802630043186
det	är	1404	18649	0.07528553809855756
är	också	15	5976	0.0025100401606425703
också	tillåtet	0	1108	0	*backoff:	2.046651371358751e-06
tillåtet	om	0	2	0	*backoff:	0.007807974981733637
om	du	176	7630	0.023066841415465267
du	vill	87	3609	0.024106400665004156
vill	söka	0	991	0	*backoff:	0.0001893152518506845
söka	olika	0	185	0	*backoff:	4.093302742717503e-05
olika	kurser	0	40	0	*backoff:	1.0233256856793756e-06
kurser	</s>	0	1	0	*backoff:	0.0526050801980

1120.1162677233517

### Online prediction of words

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

To have a more accurate prediction, you will use a trigram counting function. Program this function following the model of your bigram counting function.

In [42]:
# Write your code
def trigrams(words):
    trigrams = [tuple(words[idx:idx + 3])
                for idx in range(len(words) - 2)]
    frequencies = {}
    for trigram in trigrams:
        if trigram in frequencies:
            frequencies[trigram] += 1
        else:
            frequencies[trigram] = 1
    return frequencies

In [43]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

304

#### Prediction

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [44]:
starting_text = 'De'.lower()
starting_text

'de'

Write a program to rank the five first candidates at this point. Assign these predictions in a list that you will call `current_word_predictions_1`. Note that you are starting a sentence and you can then use the bigram frequencies. Write a sorting key that will enable you to have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [45]:
cand_nbr = 5

In [72]:
# Write your code here
#<s>
current_word_predictions_1 = []
######
#sorted_dict = sorted(frequency.items(), key=lambda x: (x[1], x[0])) #Sorterar unigrams efter freq
#sorted_keys = [item[0] for item in sorted_dict]

#temp = [word for word in sorted_keys if word[:len(starting_text)] == starting_text] #Lägger till alla som börjar på starting_text
#current_word_predictions_1 = temp[:cand_nbr]
######

print(type(frequency_bigrams))

sorted_dict = sorted(frequency_bigrams.items(), key=lambda x: (-x[1], x[0])) #Sorterar unigrams efter freq
sorted_keys = [item[0] for item in sorted_dict]
temp = [y for x, y in sorted_keys if x == "<s>" and y[:len(starting_text)] == starting_text]
current_word_predictions_1 = temp[:cand_nbr]


print(current_word_predictions_1)


<class 'dict'>
['det', 'de', 'den', 'detta', 'denna']


In [73]:
current_word_predictions_1

['det', 'de', 'den', 'detta', 'denna']

Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [74]:
current_text = "Det var en ".lower()
current_text

'det var en '

Tokenize this text and return a list of tokens. Call it `tokens`.

In [75]:
# Write your code here
tokens = tokenize(current_text)

In [50]:
tokens

['det', 'var', 'en']

Write a program to propose the five next possible words ranked by frequency using a trigram model. Assign these predictions to a variable that you will call `next_word_predictions`. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [79]:
# Write your code here


#sorted_dict = sorted(frequency_bigrams.items(), key=lambda x: (-x[1], x[0])) #Sorterar unigrams efter freq
#sorted_keys = [item[0] for item in sorted_dict]
#temp = [y for x, y in sorted_keys if x == "<s>" and y[:len(starting_text)] == starting_text]
#current_word_predictions_1 = temp[:cand_nbr]

sorted_dict = sorted(frequency_trigrams.items(), key=lambda x: (-x[1], x[0])) #Sorterar trigrams efter freq och bokstavsordning
sorted_keys = [item[0] for item in sorted_dict]
#det var en
#     x   y  z  
temp = [z for x, y, z in sorted_keys if x == tokens[-2] and y == tokens[-1]]

print()
next_word_predictions = temp[:5]



<class 'dict'>



In [80]:
next_word_predictions

['stor', 'liten', 'gammal', 'av', 'god']

Finally, let us suppose that the user has typed _Det var en g_, rank the five possible candidates. Assign these predictions in a list that you will call `current_word_predictions_2`

In [81]:
current_text = "Det var en g".lower()
tokens = tokenize(current_text)

In [84]:
# Write your code here
sorted_dict = sorted(frequency_trigrams.items(), key=lambda x: (-x[1], x[0])) #Sorterar trigrams efter freq och bokstavsordning
sorted_keys = [item[0] for item in sorted_dict]
#... var en g
#     x   y  z  
temp = [z for x, y, z in sorted_keys if x == tokens[-3] and y == tokens[-2]]
current_word_predictions_2 = [w for w in temp if w[:len(tokens[-1])] == tokens[-1]]

In [85]:
current_word_predictions_2

['gammal',
 'god',
 'gång',
 'ganska',
 'gammal,',
 'glädje',
 'grann',
 'gagnlös',
 'gammaldags',
 'gast,',
 'gengångare,',
 'getabock',
 'glad',
 'glittrande',
 'glänsande',
 'god,',
 'godmodig',
 'gosse,',
 'graf',
 'grov',
 'gruva',
 'gruvarbetare',
 'gränsbo,',
 'gråvädersdag',
 'gudsförnekare,',
 'gård',
 'gås,',
 'gåva,']

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. You will include the __regular expression you used to segment the text__ and the __unigram and bigram tables__ for _Det var en gång en katt som hette Nils_ and __two other sentences__.
2. Execute the Jupyter notebook by Peter Norvig here: <a href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">https://nbviewer.jupyter.org/url/norvig.com/ipython/How to Do Things with Words.ipynb</a>. Just run all the cells and be sure that you understand the code. You will find the data here: <a href="http://norvig.com/ngrams/">http://norvig.com/ngrams/</a>.
3. In your report, after the description of your program, you will describe one experiment with Norvig's notebook and a __long string of words your will create yourself or copy from a text you like__. You will remove all the punctuation and white spaces from this string. You will set this string in lowercase letters. You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. __You will comment the segmentation results you obtained__ with the unigram and bigram models.

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is September 22, 2023. You will have only three submission attempts. The deadline for the second and third ones are one week after you are noticed of your result.